In [1]:
!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

In [3]:
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

In [4]:
import pandas as pd

In [8]:
df = pd.read_csv("/content/drive/MyDrive/ColabData/kaggle-titanic/train.csv")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
def makeDataList(csv_path):
  datalist = pd.read_csv(csv_path)
  datalist = datalist.drop(["Name", "Ticket", "Cabin"], axis=1)
  datalist = pd.get_dummies(datalist)
  datalist = datalist.fillna(-1)
  return datalist

In [26]:
datalist = makeDataList("/content/drive/MyDrive/ColabData/kaggle-titanic/train.csv")

In [27]:
datalist.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [29]:
from sklearn.model_selection import train_test_split

train_datalist, val_datalist = train_test_split(datalist, test_size=0.1, random_state=1234, shuffle=True)

In [36]:
print(train_datalist.shape)
print(val_datalist.shape)
print(train_datalist.values)
train_datalist.head()

(801, 12)
(90, 12)
[[268.   1.   3. ...   0.   0.   1.]
 [636.   1.   2. ...   0.   0.   1.]
 [474.   1.   2. ...   1.   0.   0.]
 ...
 [295.   0.   3. ...   0.   0.   1.]
 [724.   0.   2. ...   0.   0.   1.]
 [816.   0.   1. ...   0.   0.   1.]]


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
267,268,1,3,25.0,1,0,7.7750,0,1,0,0,1
635,636,1,2,28.0,0,0,13.0000,1,0,0,0,1
473,474,1,2,23.0,0,0,13.7917,1,0,1,0,0
207,208,1,3,26.0,0,0,18.7875,0,1,1,0,0
290,291,1,1,26.0,0,0,78.8500,1,0,0,0,1


In [37]:
#DataFrameからndarrayに変換
class DatasetMaker(Dataset):
  def __init__(self, datalist):
    self.input_datalist = datalist.drop(["PassengerId", "Survived"], axis=1).values.astype(np.float32)
    self.label_datalist = datalist["Survived"].values.astype(np.long)

  #特殊メソッドで演算定義?
  def __len__(self):
    return len(self.input_datalist)

  def __getitem__(self, index):
    inputs = self.input_datalist[index]
    labels = self.label_datalist[index]
    return inputs, labels

In [41]:
dataset = DatasetMaker(datalist)
print(dataset.__len__())
print(dataset.__getitem__(index=0)[0])
print(dataset.__getitem__(index=0)[1])

891
[ 3.   22.    1.    0.    7.25  0.    1.    0.    0.    1.  ]
0


<ipython-input-37-3b54177249bb>:4: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.label_datalist = datalist["Survived"].values.astype(np.long)
